# SPARC-4 mini-pipeline tools
## Polarimetry time series

This notebook shows an example for obtaining a polarimetric time series

* Eder Martioli, LNA 13 Jun 2023

In [1]:
import os, sys
path_parent = os.path.dirname(os.getcwd())
# path_parent = "/Volumes/Samsung_T5/sparc4-pipeline/"
sys.path.append(path_parent) 

import glob

import sparc4_pipeline_lib as s4pipelib
import sparc4_utils as s4utils
import sparc4_product_plots as s4plt

### User inputs

In [2]:
# set an object ID matching the ID in the image header keyword 'OBJECT'
OBJECTID = "HD111579"
OBJECT_SUFFIX = "hd111_l2"

# set night directory name
NIGHTDIR = '20230503'
# set raw data root directory
DATADIR =  "/Volumes/Samsung_T5/Data/SPARC4/minidata/"
# set reduced data root directory
REDUCEDDIR = "/Volumes/Samsung_T5/Data/SPARC4/minidata/reduced/"

# set SPARC4 channel
CHANNEL = 1  # 1, 2, 3 or 4

# whether or not to force reduction even if product already exists
FORCE = True

# get SPARC4 pipeline parameters
p = s4pipelib.init_s4_p(datadir=DATADIR,
                        reducedir=REDUCEDDIR,
                        nightdir=NIGHTDIR,
                        channels="{}".format(CHANNEL),
                        print_report=False)

# Calculate POLARIMETRY time series

In [3]:
# set wild card to identify polarimetric sequence
l2seq_wild_card = "_{}_polar.fits".format(OBJECT_SUFFIX)

# produce list of reduced polarimetry frames in the sequence
l2seq = glob.glob("{}/*{}".format(p['reduce_directories'][CHANNEL-1], l2seq_wild_card))

# set suffix for output time series filename
ts_suffix = "{}_s4c{}_{}_L4".format(NIGHTDIR,p['CHANNELS'][CHANNEL-1],OBJECTID.replace(" ",""))

# create polar time series product
polar_ts_L4product = s4pipelib.polar_time_series(l2seq,
                                                 reduce_dir=REDUCEDDIR,
                                                 ts_suffix=ts_suffix,
                                                 aperture_index=p['APERTURE_INDEX_FOR_PHOTOMETRY_IN_POLAR'],
                                                 min_aperture=p['MIN_APERTURE_FOR_POLARIMETRY'],
                                                 max_aperture=p['MAX_APERTURE_FOR_POLARIMETRY'],
                                                 force=FORCE)

# Check the time series data in the FITS file

In [15]:
import astropy.io.fits as fits
# open the polarimetric time series product:
hdul = fits.open(polar_ts_L4product)
# print info, where each extension contains the time series data for one source. 
hdul.info()

Filename: /Volumes/Samsung_T5/Data/SPARC4/minidata/reduced/20230503_s4c1_HD111579_L4_polar_ts.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      26   ()      
  1  POLAR_TIMESERIES_00000    1 BinTableHDU     59   1R x 25C   [D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D]   
  2  POLAR_TIMESERIES_00001    1 BinTableHDU     59   1R x 25C   [D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D]   
  3  POLAR_TIMESERIES_00002    1 BinTableHDU     59   1R x 25C   [D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D]   
  4  POLAR_TIMESERIES_00003    1 BinTableHDU     59   1R x 25C   [D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D]   
  5  POLAR_TIMESERIES_00004    1 BinTableHDU     59   1R x 25C   [D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D]   


In [16]:
# Below is to check the time series data for the first source
hdul["POLAR_TIMESERIES_00000"].data

FITS_rec([(2460068.5201095, -13.72050512, 0.00035426, 4.45102879, 489.42837751, 514.32285393, 492.68593614, 556.96670452, 0.03773501, 0.00031304, -0.0497265, 0.00031298, nan, nan, 0.0624232, 0.000313, 153.59654485, 0.14340636, 1.04154518, nan, 0., 0., 16., 5., 0.549838)],
         dtype=(numpy.record, [('TIME', '>f8'), ('MAG', '>f8'), ('EMAG', '>f8'), ('FWHM', '>f8'), ('X1', '>f8'), ('Y1', '>f8'), ('X2', '>f8'), ('Y2', '>f8'), ('Q', '>f8'), ('EQ', '>f8'), ('U', '>f8'), ('EU', '>f8'), ('V', '>f8'), ('EV', '>f8'), ('P', '>f8'), ('EP', '>f8'), ('THETA', '>f8'), ('ETHETA', '>f8'), ('K', '>f8'), ('EK', '>f8'), ('ZERO', '>f8'), ('EZERO', '>f8'), ('NOBS', '>f8'), ('NPAR', '>f8'), ('CHI2', '>f8')]))

In [17]:
# Below is an example on how to get times, polarizations and uncertainties for the time series.
bjd = hdul["POLAR_TIMESERIES_00000"].data['TIME']
pol = hdul["POLAR_TIMESERIES_00000"].data['P'] 
polerr = hdul["POLAR_TIMESERIES_00000"].data['EP'] 